In [ ]:
import os, sys
from scipy import stats
import numpy as np
import pandas as pd

from   scipy import stats
#-- for ANOVA
import statsmodels.api as sm
from   statsmodels.formula.api import ols

#-- for Tukey
from statsmodels.stats.multicomp import MultiComparison

sys.path.insert(1, '../src/')
from stat_lib import *

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Comparando-se o mesmo grupo muitas vezes

  - ANOVA
  - Tukey
  - Dunnett

### ANOVA - Teste de Hipótese de Análise de Variâncias

H0 - hipótese nula:
  - todos grupos têm médias e desvios padrões amostrais próximos ou iguais
  - todas as variáveis randômicas são obtidas por sorteio de uma mesma distribuição
  
Ha - hipótese alternativa:
  - ao menos um grupo tem média e desvio padrão amostral diferentes dos outros
  - ao menos uma variável randômicas foi obtida por sorteio de uma outra distribuição

https://en.wikipedia.org/wiki/One-way_analysis_of_variance

### Novos exemplos
  - dadas 5 hipotéticas amostras
  - com media 140 e variando com delMU
  - com SSD 10, variando com delSSD

In [ ]:
samp_list=[]; mu_list = []; ssd_list = []
N   = 30
n_samp = 5

MU0 = 140; delMUs = [0, -.5, +1, -20, -30]
SSD0 = 10; delSSD = [0, -.2, +.2, -1, +2]

for i in range(n_samp):
    MU = MU0 + delMUs[i]
    SSD = SSD0 + delSSD[i]

    samples = np.random.normal(loc=MU, scale=SSD, size=N)
    
    samp_list.append(samples)

    mu_list.append(np.mean(samp_list[i]))
    ssd_list.append(np.std(samp_list[i]))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

seqx = np.linspace(70, 180, 100)
colors = ['red', 'blue', 'green', 'brown', 'black']

for i in range(n_samp):

    samples = samp_list[i]
    color = colors[i]

    label = f"{color} {mu_list[i]:.1f} ({ssd_list[i]:.1f})"
    # ax = sns.histplot(samples, stat='density', color=color, alpha=.2, label=label, ax=ax)
    sns.rugplot(samples, color=color, alpha=0.4, label=label, ax=ax)

    plt.vlines(mu_list[i], 0, 0.06, color=color)

    normal_pdf = stats.norm.pdf(seqx, mu_list[i], ssd_list[i])
    sns.lineplot(x=seqx, y=normal_pdf, color=color)

title = 'Distribuições'
plt.legend()
plt.grid()
plt.ylim(0, 0.06)
plt.title(title);


### As distribuições são nomais? teste de Shaprio-Wilkis

In [ ]:
for i in range(n_samp):
    ret, text, text_stat, stat, pvalue = calc_normalidade_SWT(samp_list[i])
    print(text, '\n', text_stat, '\n')

### Alguma distribuição tem média diferente? one-way ANOVA

In [ ]:
ret, text, text_stat, stat, pvalue = test_one_way_ANOVA5(samp_list[0],samp_list[1],samp_list[2],samp_list[3],samp_list[4])
text, text_stat, stat

## Qual grupo é diferente?

### Tukey test - Post-hoc test 

É chamado de test de Tukey, ou método de Tukey, ou teste de significância honesta de Tukey
 
**POST-HOC** - depois disto

**ANOVA** diz se as distribuições são diferentes mas não diz qual e quanto.

https://en.wikipedia.org/wiki/Tukey%27s_range_test

In [ ]:
plt.figure(figsize=(12,8), dpi=300)

cardata = MultiComparison(df.val, df.group)
results = cardata.tukeyhsd()

title  = "Tukey test: multiple comparisons between all pairs"

results.plot_simultaneous()
plt.title(title);

### Combinações

comb(5 2) = 5! / 3! 2! = 5 * 4 / 2 = 20 / 2 = 10

In [ ]:
for i in range(samples-1):
    for j in range(i+1, samples):
        print(i, j)

In [ ]:
results.meandiffs

In [ ]:
results.confint

In [ ]:
results.pvalues

In [ ]:
results.summary()

### Dunnett test

  - Um único controle
  - Múltiplos cases
  
Este é o caso quando queremos fazer um experimento de processo anti-inflamatório:
  - Temos uma cultura de células em PBS
  - Adicionamos um sinal inflmatório e medimos TNF após 30 min
  - Após uma hora adicionamos dexametasona e medimos TNF-A após 2 horas
  - Repetimos o experimento acima adicionando uma dada droga e medindo o TNF-A após 2 horas
  
    - Controle: PBS
    - Case: controle positivo - inflamação
    - Case: controle negativo - dexametazona
    - Cases: mais 3 cases com 3 outras drogas
  

In [ ]:
samps=[]; mus = []; sdvs = []
N   = 12
hline = 0.03; colors = ['red', 'blue', 'green', 'brown', 'black', 'gold']
sampNames = ['control', 'ctrl-pos', 'ctrl-neg', 'drg1', 'drg2', 'drg3']

# TNF-A ... pmol
mus  = [20, 400, 40, 420, 240, 90]
sdvs = [10, 25,  10,  25,  20, 11]
samples = len(mus)

for i in range(samples):
    samps.append(np.random.normal(loc=mus[i], scale=sdvs[i], size=N))


fig = plt.figure(figsize=(12, 6))
seqx = np.linspace(-20, 480, 100)

for i in range(samples):
    if i == 0:
        ax  = sns.distplot(samps[i], kde=False, rug=True, norm_hist=True, label=sampNames[i], color=colors[i], rug_kws={"color": 'blue', "alpha": .1,})
    else:
        ret = sns.distplot(samps[i], kde=False, rug=True, norm_hist=True, label=sampNames[i], color=colors[i], rug_kws={"color": 'red',  "alpha": .1,}, ax=ax)

    plt.vlines(mus[i], 0, hline, color = 'navy')
    
    sns.lineplot(seqx, stats.norm.pdf(seqx, mus[i], sdvs[i]), color=colors[i])

title = 'Distribuições'
plt.title(title)
plt.legend;

### Barplot

In [ ]:
df2 = pd.DataFrame([samps[0], [sampNames[0]]*N]).T
df2.columns = ['val', 'group']

for i in range(1,samples):
    dfa = pd.DataFrame([samps[i], [sampNames[i]]*N]).T
    dfa.columns = ['val', 'group']
    
    df2 = df2.append(dfa)

df2.shape, df2.group.unique()

In [ ]:
plt.figure(figsize=(12,6))
ci = 95

ax = sns.barplot(x="group", y="val", data=df2, saturation=0.6, palette=colors, ci=ci)

for i in range(3):
    mu = np.mean(samps[i])
    plt.hlines(mu, 0, 6, color = colors[i])

plt.title("Teste de 3 drogas x controle pos e neg")
plt.ylabel('TNF-A (pmol)')
plt.xlabel('')

In [ ]:
sns.boxplot(x="group", y="val", data=df2, saturation=0.6, palette=colors);

### 3 gráficos juntos

In [ ]:
ci = 95

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20,10), sharey=True)

ret = sns.barplot(x="group", y="val", data=df2, saturation=0.6, palette=colors, ci=ci, ax=ax[0])
    
ax[0].set_ylabel('TNF-A (pmol)')
ax[0].set_xlabel('')
ax[0].set_title("Distribuições com n = %d"%(N))

seqx = np.linspace(-20, 480, 100)

for i in range(samples):
    retQ = sns.distplot(samps[i], kde=True, rug=True, norm_hist=True, label=sampNames[i], color=colors[i], vertical=True, rug_kws={"color": 'blue', "alpha": .1,}, ax=ax[1])
    # ax[1].vlines(mus[i], 0, hline, color = 'navy')
    
    # sns.lineplot(seqx, stats.norm.pdf(seqx, mus[i], sdvs[i]), color=colors[i], ax=ax[1])

ax[1].legend()

sns.boxplot(x="group", y="val", data=df2, saturation=0.6, palette=colors, ax=ax[2])
ax[2].set_ylabel('')
ax[2].set_xlabel('')
ax[2].set_title("box-plot");

In [ ]:
df2.group.unique()

In [ ]:
df2.head()

In [ ]:
try:
    os.mkdir('../tmp')
except:
    pass

df2.to_csv('../tmp/table.tsv', sep='\t', index=False)

In [ ]:
df3 = pd.read_csv('../tmp/table.tsv', sep='\t')
print(df3.shape, N*samples)
df3.head(3)

In [ ]:
df3.tail(3)

### Interfaceando R

In [ ]:
os.system("Rscript calcd.R")

In [ ]:
fname = 'dunnet_result.tsv'
filefull = os.path.join('../tmp/', fname)
    
dfd = pd.read_csv(filefull, sep='\t')
dfd

In [ ]:
gamma = stats.t.ppf(0.025, N-1)
gamma

In [ ]:
ci = 95

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20,10), sharey=True)

ret = sns.barplot(x="group", y="val", data=df2, saturation=0.6, palette=colors, ci=ci, ax=ax[0])

ax[0].set_ylabel('TNF-A (pmol)')
ax[0].set_xlabel('')
ax[0].set_title("Distribuições com n = %d\nDunnett-test, comparações contra controle positivo"%(N))

y1 = 380; dely = 32

#-- barras de erros
for j in [0,1,2,3, 4]:
    y1 += dely
    x1 = 0 if j==0 else j+1
    xt = j if j<=1 else 1.5

    ax[0].hlines(y=y1, xmin=1, xmax=x1, colors='black')
    text = 'pval %.1e'%(dfd.iloc[j].pvalue)

    ax[0].text(x=xt, y=y1+10, s=text)

seqx = np.linspace(-20, 480, 100)

for i in range(samples):
    retQ = sns.distplot(samps[i], kde=True, rug=True, norm_hist=True, label=sampNames[i], color=colors[i], vertical=True, rug_kws={"color": 'blue', "alpha": .1,}, ax=ax[1])

ax[1].legend()
xerror = 0.02; delxerror = 0.0025

#--- medias e barra de erro na distribuição
for i in range(samples):
    mu    = np.mean(samps[i])
    SEM   = np.std(samps[i])/np.sqrt(N)
    error = gamma * SEM

    xerror += delxerror

    ax[1].hlines(y=mu, xmin=0, xmax=0.04, colors='black')
    ax[1].vlines(x=xerror, ymin=mu+error, ymax=mu-error, colors='black')

ax[1].set_ylabel('')
ax[1].set_xlabel('percentagem (%)')
ax[1].set_title("distribuições");


sns.boxplot(x="group", y="val", data=df2, saturation=0.6, palette=colors, ax=ax[2])
ax[2].set_ylabel('')
ax[2].set_xlabel('')
ax[2].set_title("box-plot");